# Guidance on wind shear for aviation
### This analysis will identify risk regions and issue shear warning accordingly.

Username: 

Date: 

In [ ]:
# Pre-loading Python libraries needed for this notebook

import intake
import geoviews as gv

from sidecar import Sidecar
from holoviews.operation.datashader import regrid
from geoviews.feature import coastline as coastlines

gv.extension('bokeh', logo=False)
from geoviews import opts
opts.defaults(opts.Polygons(line_color='orange', line_width=3, fill_color='orange', fill_alpha=0.2))

import sys
# Suppress warnings
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

### Calculate wind shear from the Mogreps-G model runs
Firstly, I'm going to find the relevant data from the Met Office global ensemble forecast, Mogreps-G.

In [ ]:
intake.gui

The catalogue above shows the data available from Mogreps-G. 
<br>I need wind speed and direction to calculate shear, and since I am working with an aviation customer vertical pressure levels are most useful.

In [ ]:
direction = intake.cat.mo_aws_earth.mogreps_g.wind_from_direction_at_pressure.read()
speed = intake.cat.mo_aws_earth.mogreps_g.wind_speed_at_pressure.read()

I can calculate the shear between consecutive pressure levels using the analysis tools created by one of my colleagues. 
<br>No actual processing of the data will occur until I need to make my plots.

In [ ]:
from opscentretools import sheartools
shear = sheartools.calculate_shear(speed, direction)
shear

This is a preview of the whole processed dataset, including every forecast reference time and period from the last seven days of model runs.

This represents multiple terrabytes of data.

In [ ]:
from opscentretools import datatools 
datasize = datatools.estimate_cube_size(shear)
datasize

### Plot of windshear

I want to visualise my data to understand what the wind shear looks like.

First I want to see the field at a single pressure level (98750 Pa) for the latest T+00 forecast.

In [ ]:
from opscentretools import plotting
latest_T00_P98750 = shear[0, -4, 0, 0]
shear_T00_P98750_plot = plotting.interactive_plot(latest_T00_P98750, coastlines=True)

In [ ]:
shear_T00_P98750_plot

This plot is useful to see what the wind shear looks like but I would like to compare it at different pressure levels.

In [ ]:
latest_T00 = shear[0, -4, 0, 0:15]
shear_T00_plot = plotting.interactive_plot(latest_T00, coastlines=True)

In [ ]:
shear_T00_plot

## Annotating the plot

There appears to be an area of high wind shear in the North Atlantic. I'm going to add a weather warning tool to my plot so I can highlight it.

In [ ]:
orange_warning_pen, orange_warning = plotting.warning_tool("orange")

In [ ]:
shear_T00_plot * orange_warning

The annotations I make on my plot can be recorded and reused. I want to check what my warning looks like with just a map of coastlines.

In [ ]:
wind_shear_warning = orange_warning_pen.element

In [ ]:
with Sidecar():
    display(wind_shear_warning * coastlines)

### Consult with the chief

I would like to check my findings with the Chief Forecaster to make sure they fit with today's forecasting narrative.

In [ ]:
wind_shear_for_chief = shear_T00_plot.select(pressure=98750) * orange_warning * wind_shear_warning

I can generate a web link that means anyone can my annotated plot from any device.
<br>This is the link I will share with the Chief for their feedback.

In [ ]:
from opscentretools import annotable
annotable.webapp(wind_shear_for_chief, websocket_origin='pangeo-dev.informaticslab.co.uk')